In [1]:
# Python routine to read position of motion controller and resulting current from ammeter

#Setup block. Run this once to import the libraries and files we will need.
%pylab qt
import keyboard
import matplotlib.pyplot as plt # For plotting
from matplotlib.animation import FuncAnimation # For real time display
import winsound                 # For making the computer beep
from time import time, sleep, localtime, strftime  #Timekeeping functions
import csv                      # For writing the data to a file (easier than writing with pandas when appending to file)
import pandas as pd             # For reading the data from a file (easier than reading with csv)

import pyvisa as visa           # For communicating with instruments
import pyvisa.constants         # needed to access comm buffers

import scipy.special as ss      # For plotting Fresnel Integrals

Populating the interactive namespace from numpy and matplotlib


In [2]:
#Set up resources

rm = visa.ResourceManager()
print("Here are the resources in use: ")
print(rm.list_resources())
resources = rm.list_resources()

# Set the names to verify in identification
ammeter_name = "MODEL 6485"
mo_cont_name = "MM3000"
fields = ['Position', 'Current']


# Connect to Motion Controller "mo_cont"
mo_cont = rm.open_resource('ASRL1::INSTR')
mo_cont.timeout = 4000
print("\nOpening resource ", mo_cont)
mo_cont.read_termination = '\r\n'
mo_cont.write_termination = '\r'
mo_cont.flush(pyvisa.constants.BufferOperation.discard_receive_buffer)
mo_cont.flush(pyvisa.constants.BufferOperation.discard_write_buffer)
sleep(2)
    
try:
    return_str = mo_cont.query('VE') # checks the version history which includes the name of the device

    sleep(0.5)
    print('VE response: ',return_str)
    #Check if usable resources are multimeters
    #   if they are, assign the ammeter first
    if mo_cont_name in return_str:
        print("\nMotion Controller found: ", return_str)
        mo_cont.timeout = 3000
        mo_cont.close()

except Exception as E:
    errorname = str(E).rsplit(' ')[0]
    print(" unavailable; ", E)
    mo_cont.close()



res_list=['ASRL1::INSTR']
ammeter = None

#Look through other ports to detect ammeter
print("Opening them in turn and asking who they are: ")
for i in resources:
    if (i in res_list):  # Don't look at instrument already found
        continue
    print("Resource: ", i, end='')
    
    #Check if resources are usable
    
    res_item = rm.open_resource(i)
    res_item.timeout = 1000
    res_item.baud_rate = 9600
    print("\nOpening resource: ", res_item)
    res_item.read_termination = '\n'
    res_item.write_termination = '\n'
    res_item.flush(pyvisa.constants.BufferOperation.discard_receive_buffer)
    res_item.flush(pyvisa.constants.BufferOperation.discard_write_buffer)
    sleep(2)
    print("Asking for identification...")
    
    try:
        return_str = res_item.query('*IDN?') # checks the name of the device

            
        print('*IDN? response: ',return_str)
        #Check if usable resources are multimeters
        #   if they are, assign the ammeter first
        if ammeter_name in return_str:
            ammeter = res_item
            res_list.append(ammeter)
            print("\nAmmeter found: ", return_str)
            res_item.close()
            continue
            
    
    except Exception as E:
        errorname = str(E).rsplit(' ')[0]
        print(" unavailable; ", E)
        res_item.close()
        continue



Here are the resources in use: 
('ASRL1::INSTR', 'ASRL5::INSTR')

Opening resource  SerialInstrument at ASRL1::INSTR
VE response:  Newport Corp. MM3000 Version 3.0 

Motion Controller found:  Newport Corp. MM3000 Version 3.0 
Opening them in turn and asking who they are: 
Resource:  ASRL5::INSTR
Opening resource:  SerialInstrument at ASRL5::INSTR
Asking for identification...
*IDN? response:  KEITHLEY INSTRUMENTS INC.,MODEL 6485,1243641,B04   Jun 20 2006 16:11:55/A02  /G

Ammeter found:  KEITHLEY INSTRUMENTS INC.,MODEL 6485,1243641,B04   Jun 20 2006 16:11:55/A02  /G


In [44]:
# Initial parameters
strt_pos_x = -500 # μm
end_pos_x = 1000 # μm
step_size = 10 # μm
delay_s = 0.2 # Dwell time between measurements
delay_m = 1   # Time for allowing devices to register commands
delay_l = 7   # Time for allowing devices complete slow commands

# Open the ports that have been identified for each device
ammeter.open()
mo_cont.open()
sleep(delay_m)
mo_cont.flush(pyvisa.constants.BufferOperation.discard_receive_buffer)
mo_cont.flush(pyvisa.constants.BufferOperation.discard_write_buffer)
ammeter.flush(pyvisa.constants.BufferOperation.discard_receive_buffer)

# Setup the file for saving data
now = strftime("%d%b%y_%I-%M-%S",localtime()) #grab the current time and put it in the filename
filename = "FresnelDiffraction_" + now + "A.csv"
print("Data will be written to " + filename)

f_out = open(filename, 'w', newline='')
datafile = csv.writer(f_out, delimiter=',')
datafile.writerow(fields) #Write header to csv

# Convert from Motor steps to mm
def StepsToMM(steps):
    return (float(steps)*0.0000555) # Convert steps to μm (Assuming step amount is 0.0000555 mm)

# Initialize motion controller assuming the use of MFN25CC Motor
def InitDevices():
    try:
        mo_cont.write('1US0.0555um')
        mo_cont.write('1UUmm')
        mo_cont.write('1UV0.3')
        mo_cont.write('1UA1.2')
        if(strt_pos_x < 0):
            mo_cont.write(f'1UP-{abs(strt_pos_x/1000)}')
        else:
            mo_cont.write(f'1UP+{abs(strt_pos_x/1000)}')
        mo_cont.write('1FE+500')
        mo_cont.write('1IL+100')
        mo_cont.write('1KP+1500')
        mo_cont.write('1KD+5000')
        mo_cont.write('1KI+100')
        ammeter.write('*RST')
        sleep(delay_m)
        ammeter.write(':sens:curr:rang:auto on')
        #ammeter.write(':sens:curr:rang 2e-5')
        ammeter.write(':curr:nplc 6')
        ammeter.write(':syst:zch on')
        sleep(delay_m)
        ammeter.write(':syst:azer:stat off')
        ammeter.write(':form:elem read')
        ammeter.write(':syst:zch off')
        ammeter.write(':syst:zcor on')
        
        
    except:
        mo_cont.flush(pyvisa.constants.BufferOperation.discard_receive_buffer)
        mo_cont.flush(pyvisa.constants.BufferOperation.discard_write_buffer)
        print("Error")
    print("Devices Initialized")

# Move motor by stepSize
def IteratePos():
    try:
        mo_cont.write(f'1UR+{step_size/1000}')
    except:
        mo_cont.flush(pyvisa.constants.BufferOperation.discard_receive_buffer)
        mo_cont.flush(pyvisa.constants.BufferOperation.discard_write_buffer)
        print("Error")
        
# Present length of time in seconds in more readable format
def FormatTime(t):
    if(t < 1):
        return "0 secs"
    output = ""
    if(int(int(t)/3600) > 0):
        output += f"{int(int(t)/3600):04d} hrs "
    else:
        output += ""
    if(int((int(t) % 3600) / 60) > 0):
        output += f"{int((int(t) % 3600) / 60):04d} mins "
    else:
        output += ""
    output += f"{(t - int((int(t)/3600))*3600 - int((int(t) % 3600) / 60)*60):02.2f} secs "
    return output

InitDevices()
sleep(delay_l)
    
#initialize the animated plot
fig, ax = plt.subplots()
plt.xlabel("Position (mm)")
plt.ylabel("Current (A)")
xdata, ydata = [], []
ln, = plot([], [], '-')


print("Starting the scan")
print("Live plot may appear in the task bar at bottom of screen")
sleep(0.25)
startTime = time()
mo_cont.flush(pyvisa.constants.BufferOperation.discard_receive_buffer)
mo_cont.flush(pyvisa.constants.BufferOperation.discard_write_buffer)
#live update of plot here
count = 0
done_flag = False
def update(count, done_flag):
    while(done_flag == False): 
        
        # Fetch the data from motion controller and ammeter
        curr_A = -float(ammeter.query(':READ?').rsplit('A')[0])
        pos_x = StepsToMM(int(str(mo_cont.query('1TPI')).rsplit(' ')[0]))
        IteratePos()
      
        # Add data to graph vars and file
        xdata.append(pos_x)
        ydata.append(curr_A)
        datafile.writerow([pos_x, curr_A])
        
        # Report of elapsed time and remaining time
        elapsedTime = time() - startTime
        print(f"Current Position: {pos_x:07.2f}mm Elapsed Time: {FormatTime(elapsedTime)} Est Remaining Time: {FormatTime(elapsedTime/((pos_x-strt_pos_x)/(end_pos_x-strt_pos_x))-elapsedTime)}                       ", end="\r", flush=True)
    
        # Format the animated graph
        ln.set_data(xdata, ydata)
        ax.relim() 
        ax.autoscale_view()
        plt.xlim([strt_pos_x/1000, end_pos_x/1000])
        title("Current vs. Position")
        count += 1
        sleep(delay_s)  #dwell time at each data point.  Below 0.2 doesn't leave time to plot
        
        if pos_x > end_pos_x: 
            # When final position has been reached
            print(flush = False)
            f_out.close()
            print("Finished gathering data")
            done_flag = True
            sleep(delay_s)
            ani.event_source.stop()
            plt.close()
            figure()
            title("Current vs. Position")
            plt.xlabel("Position (mm)")
            plt.ylabel("Current (A)")
            plot(xdata,ydata,'-')
            
            
            
        else:
            return ln,

# Define and run the animation
ani = FuncAnimation(fig, update,interval = 500,fargs = [done_flag])        

# Show final plots
plt.show()

Data will be written to FresnelDiffraction_23Jun22_04-27-06A.csv
Devices Initialized
Starting the scan
Live plot may appear in the task bar at bottom of screen


In [38]:
r_s = 0.340
r_o = 0.234

In [41]:
max_idx = 0
max_val = 0
for i in range(len(ydata)):
    if ydata[i] > max_val:
        max_val = ydata[i]
        max_idx = i
        
sum = 0;
for i in range(max_idx, len(ydata)):
    sum += ydata[i]
    
xdata = array(xdata)
ydata = array(ydata)
    
i_0 = sum / (len(ydata)-max_idx)
u_exp=xdata*sqrt((2/(632.8*1E-9))*(1/r_s + 1/r_o))/(1000)

u_mod=linspace(-10,10,10000)
# figure()
# plot(u_mod[5000:10000], ss.fresnel(u_mod[5000:10000])[1])
# plot(u_mod[5000:10000], ss.fresnel(u_mod[5000:10000])[0])
# figure()
# grid()
# plot(ss.fresnel(u_mod)[1], ss.fresnel(u_mod)[0])
# ufew = arange(-5,5)
# title("Cornu Spiral, with u = [-5:5] plotted as red dots")
# plot(ss.fresnel(ufew)[1],ss.fresnel(ufew)[0],'or')
# xlabel("C(u)")
# ylabel("S(u)")
figure()
title("Knife Edge")
plot(u,square(ss.fresnel(u_mod)[1]-(-0.5))+square(ss.fresnel(u_mod)[0]-(-0.5)),'-')
plot(u_exp, ydata/i_0 * 2, '-r')
xlabel("u")
ylabel("Intensity")


Text(0, 0.5, 'Intensity')